In [12]:
import face_rec

In [13]:
import pandas as pd
import numpy as np
import cv2

In [14]:
face_rec.r.ping()

True

#### Step-1: Extract Data from database

In [15]:
name = 'academy:register'
retrive_dict= face_rec.r.hgetall(name)
retrive_series = pd.Series(retrive_dict)
retrive_series = retrive_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrive_series.index
index = list(map(lambda x: x.decode(), index))
retrive_series.index = index
retrive_df =  retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role','facial_features']
retrive_df[['Name','Role']] = retrive_df['name_role'].apply(lambda x: x.split('@')).apply(pd.Series)
retrive_df

,name_role,facial_features,Name,Role
0,Ayush_PC24@Student,"[-1.1310793, 0.22157845, -0.8644012, 0.3219679...",Ayush_PC24,Student
1,Prabhanjan_PC19@Student,"[-0.13739689, 0.59309655, 0.6826751, 0.8627867...",Prabhanjan_PC19,Student
2,Anish_PA70@Student,"[-0.92406213, -0.71368057, -0.23641296, -0.154...",Anish_PA70,Student
3,Barack Obama@Teacher,"[0.94841325, -0.5583419, 0.09482177, -1.279714...",Barack Obama,Teacher
4,Angelina Jolie@Student,"[0.43206447, 1.075011, -0.1285299, -1.8333918,...",Angelina Jolie,Student
5,Morgan Freeman@Teacher,"[0.48527828, -0.6193287, -0.40698832, 1.068717...",Morgan Freeman,Teacher
6,Chris Evans@Student,"[0.7200023, -1.4684912, -0.7778238, -1.4601221...",Chris Evans,Student
7,Scarlett Johansson@Student,"[0.3408678, -0.5793298, -1.7011663, 0.35731304...",Scarlett Johansson,Student


### Step-2: Get Real - Time Prediction

In [18]:
cap = cv2.VideoCapture(0) # default , # 1 for external camera

while True:
    ret, frame = cap.read()
    
    if ret == False:
        break
        
    pred_frame = face_rec.face_prediction(frame,retrive_df,'facial_features',['Name','Role'],thresh=0.5)
    
    #cv2.imshow('frame',frame)
    cv2.imshow('prediciton',pred_frame)
    
    if cv2.waitKey(1) == 27: # if i press esc button this condition will get trigged
        break
        
cv2.destroyAllWindows()
cap.release()

2024-11-18 12:08:59.837 python[10893:3216014] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_bundleIdentifierWithReply:) block performed very slowly (12.53 secs).
1   HIToolbox                           0x00007ff814fcc0c2 _ZN15MenuBarInstance22RemoveAutoShowObserverEv + 30
2   HIToolbox                           0x00007ff815008638 _ZL17BroadcastInternaljPvh + 167
3   SkyLight                            0x00007ff80fca323d _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 1030
4   SkyLight                            0x00007ff80fca17f3 CGSSnarfAndDispatchDatagrams + 1055
5   SkyLight                            0x00007ff80ffb0686 SLSGetNextEventRecordInternal + 284
6   SkyLight                            0x00007ff80fdccaec SLEventCreateNextEvent + 9
7   HIToolbox                           0x00007ff814fd9156 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 45
8   HI

### Delete an Entry


In [26]:
import face_rec

In [5]:
# Function to delete entry from Redis based on name and role
def delete_entry_from_redis(name, role):
    """
    Deletes an entry from the Redis database based on the provided name and role.
    """
    key = f'{name}@{role}'  # Create the unique key based on name and role
    
    # Check if the entry exists
    if face_rec.r.hexists('academy:register', key):
        face_rec.r.hdel('academy:register', key)
        return True
    else:
        return False


In [6]:
# Example usage of the delete function
if __name__ == "__main__":
    name_to_delete = input("Enter the name of the person to delete: ")
    role_to_delete = input("Enter the role of the person to delete: ")

    if delete_entry_from_redis(name_to_delete, role_to_delete):
        print(f"Entry for {name_to_delete} ({role_to_delete}) has been deleted from Redis.")
    else:
        print(f"No entry found for {name_to_delete} ({role_to_delete}).")

Enter the name of the person to delete:  Ayush_PC24
Enter the role of the person to delete:  Student


Entry for Ayush_PC24 (Student) has been deleted from Redis.
